Copyright (c) 2023, SAS Institute Inc., Cary, NC, USA. All Rights Reserved. SPDX-License-Identifier: Apache-2.0



# Python Container Runtime Scoring
*** 
[SAS Model Manager](https://www.sas.com/en_us/software/model-manager.html) can deploy Python models into a Python container. This container deployment has these benefits:
* written in pure Python, to enable easy debugging and data handling
* scalable, to meet traffic needs
* portable, to allow movement between environments
* standardized, to leverage common tooling
* independently executable, to be used outside of SAS Viya

This notebook walks you through leveraging a Python container for scoring. But first you must perform these steps:

1. Use the SAS Viya CLI to configure a container publishing destination, such as a container registry in Docker, Amazon Web Services (AWS), Google Cloud Platform (GCP), or Azure. Examples of commands for creating each destination can be found [here](https://github.com/sassoftware/model-management-resources/tree/main/configuration/destinations).
2. Publish a Python model to container publishing destination from SAS Model Manager. More information about publishing Python models can be found [here](https://documentation.sas.com/?cdcId=mdlmgrcdc&cdcVersion=default&docsetId=mdlmgrug&docsetTarget=n04i7s6bdu7ilgn1e350am3byuxx.htm#p1e2r8l1runn7tn1iz4sxcvtswj6).
3. Pull the container from the container registry and run it. The following command can be used to pull and run the container locally using Docker on the specified port:
   
   `docker run -tid -p port-number:8080 image-id`
    
After the container is running, it can be accessed via the Rest API. The following code leverages the requests package in Python to interact with the container via the Rest API. Using Python, import the required packages. 

In [ ]:
import requests
import json

Specify the host and port number. Containers running locally can use *localhost*.

In [ ]:
host = input('Host: ')
port = input('Port Number: ')
model_name = input('Model Name: ')

Prepate your data. The Python container expects data in a CSV format. The following is an example of formatted data for a model using the [HMEQ data set](https://github.com/sassoftware/model-management-resources/blob/main/samples/Data/hmeq.csv). 

In [ ]:
csvData = \
 '"REASON","JOB","YOJ","DEROG","DELINQ","CLAGE","NINQ","CLNO","DEBTINC","MORTDUE","LOAN","VALUE"\r' + \
 '"HomeImp", "Other" , 3, 0, 2, 65.8,  1,  2, 50, 1000, 2000, 500000\r' + \
 '"HomeImp", "Sales" , 6, 0, 2, 32.5,  4,  4, 20, 10000, 20000, 40000\r' + \
 '"HomeImp", "Office", 5, 0, 2, 121.1, 2,  8, 13, 30000, 60000, 80000\r' + \
 '"HomeImp", "Other" , 2, 0, 2, 221.2, 6, 14, 10, 90000, 170000, 222003'

multipart_form_data = {'file': csvData}

To score using the container, by passing the data to the */{model_name}* endpoint using a POST request. The scoring data will be returned directly.

In [ ]:
url = 'http://' + host + ':' + port + '/' + model_name
print('Scoring Endpoint: ' , url )
r = requests.post(url, files=multipart_form_data)
print(r.json()) 

To get a two-part score execution, call the */executions* endpoint using a POST request. An execution ID that can be used to access the scoring results will be returned.

In [ ]:
url=  'http://' + host + ':' + port + '/executions'
print('Scoring Endpoint: ' , url )
r = requests.post(url, files=multipart_form_data)
print(r.json()) 
executionId = r.json()['id']

To return the results of the execution, use a GET request with the */query/execution-id-from-previous-call* endpoint. 

In [ ]:
url= 'http://' + host + ':' + port + '/query/'
r1 = requests.get(url + executionId)
print('\nOutput:\n', r1.text)

You can also check the logs for the execution using a GET request with the */execution-id-from-previous-call/log* endpoint. 

In [ ]:
r2 = requests.get(url + executionId + '/log')
print('Log:', r2.text)